# Return Stats

## Import Libraries

In [ ]:
from ipywidgets import interact, interact_manual
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import util
from EquityHedging.analytics import summary
from EquityHedging.analytics import premia_metrics as pm
from EquityHedging.analytics import returns_stats as rs
from EquityHedging.reporting import formatter as fmt, plots
import pandas as pd
import plotly.graph_objects as go


## Import liq alts data

### Imports liq alts and bmk weekly data

In [ ]:
asset_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\Historical Returns.xls', sheet_name='Historical Returns')
asset_df.columns = ['Name', 'Account Id', 'Return Type', 'Date',
       'Market Value', 'Return']
asset_ret = asset_df.pivot_table(values='Return', index='Date', columns='Name')
asset_ret /= 100
eq_fi_df = asset_ret[['Total EQ w/o Derivatives', 'Total Fixed Income']]

In [ ]:
bmk_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\bmk_data.xlsx', sheet_name='data_m', index_col=0)
bmk_df = bmk_df[['SPGSCITR Index', 'HFRXM Index','HFRIMI Index',
                 'SGSLRPEW Index', 'SGSLRPWH Index', 'SGIXTFXA Index', 'NEIXCTAT Index',
                 'HFRXAR Index', 'HFRXEH Index', 'HFRXEW Index', 'HFRXED Index',
                 'HFRXCA Index', 'HFRXEMC Index', 'HFRXCOM Index',
                 'SECTGLMA Index', 'SECTNEUT Index','SECTMSTR Index',
                 'HEDGGLMA Index', 'HEDGNAV Index','HEDGNEUT Index','HEDGCONV Index',
                 'CSLAB Index', 'CSLABED Index', 'CSLABGS Index']]
bmk_df.columns = ['S&P GSCI TR', 'HFRX Macro/CTA', 'HFRI Macro Total Index',
                  'Eqty Risk Prem', 'Eqty Risk Prem-Hedged','SGI Cross Asset Trend', 'SG Trend Index',
                  'HFRX Abs Ret', 'HFRX Eqty Hedge', 'HFRX Equal Wgtd', 'HFRX Event Driven',
                  'HFRX RV FI Conv Arb', 'HFRX EM', 'HFRX Macro Comdty',
                  'CS AllHedge Global Macro', 'CS AllHedge Eqty Mkt Neutral','CS AllHedge Multi_Strat',
                  'CS Global Macro', 'CS Hedge Fund', 'CS Eqty Mkt Neutral', 'CS Conv Arb',
                 'CS Liq Alt Beta', 'CS Event Driven', 'CS Global Strat']

bmk_df = dm.format_data(bmk_df)

In [ ]:
liq_alts_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\mgr_data_m.xlsx', sheet_name='bny',index_col=0)
liq_alts_sp = liq_alts_df[['Total Liquid Alts',  'Global Macro',  'Trend Following','Alt Risk Premia']]
liq_alts_sp = liq_alts_sp.iloc[52:,]
liq_alts_mgr = liq_alts_df[['1907 ARP EM', '1907 ARP TF', '1907 CFM ISE Risk Premia','1907 Campbell TF', '1907 III CV',
                            '1907 III Class A', '1907 Kepos RP','1907 Penso Class A', '1907 Systematica TF',
                            'ABC Reversion','Acadian Commodity AR', 'Black Pearl RP','Blueshift', 'Bridgewater Alpha',
                            'DE Shaw Oculus Fund', 'Duality','Element Capital', 'Elliott', 'One River Trend']]

In [ ]:
sub_port_list_dict = {'Global Macro': ['1907 Penso Class A', 'Bridgewater Alpha','DE Shaw Oculus Fund',
                                       'Element Capital', 'Elliott'], 
                      'Absolute Return': ['1907 ARP EM', '1907 III CV', '1907 III Class A',
                                          'ABC Reversion','Acadian Commodity AR', 'Blueshift', 'Duality'], 
                      'Trend Following':['1907 ARP TF', '1907 Campbell TF', '1907 Systematica TF', 'One River Trend'], 
                      'Alt Risk Premia':['1907 CFM ISE Risk Premia','1907 Kepos RP', 'Black Pearl RP']}
liq_alts_dict = {}
for key in sub_port_list_dict:
    temp_df = liq_alts_ret.copy()
    liq_alts_dict[key] = temp_df[sub_port_list_dict[key]]
    

In [ ]:
df_returns = util.get_weighted_hedges(df_returns, notional_weights, include_fi=True, weight_col='Liq Alts Port')


## Display Returns

In [ ]:
fmt.get_returns_styler(df_returns)

## Compute Return Statistics and Premia Metrics

In [ ]:
return_stats = rs.get_return_stats(df_returns,freq)
premia_metrics = pm.get_premia_metrics(df_returns, freq)
analytics_dict = {'return_stats':return_stats, 'premia_metrics':premia_metrics}

In [ ]:
@interact
def display_analytics(stats=['return_stats', 'premia_metrics']):
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics_dict)
    if stats == 'premia_metrics':
        return fmt.get_analytics_styler(analytics_dict, stats)

## Get Raw Scores and Normalized Hedge Metrics

In [ ]:
pm_dict = summary.get_norm_premia_metrics(df_returns)


## Display Data
To display data:
* Select which statistics you want to view **(stats)** -  Premia Metrics or Normalized Data

In [ ]:
@interact
def display_premia_metrics_or_normal( stats = ['Premia Metrics','Normalized Data']):
    if stats == 'Premia Metrics':
        return fmt.format_pm_to_normalize(pm_dict[stats])    
    else:
        return fmt.format_normalized_data(pm_dict[stats])

## Set up data to display

In [ ]:
df_norm = pm_dict['Normalized Data']
df_normal = df_norm.transpose()

#assign each strategy a symbol
symbols = plots.get_symbols(df_normal, unique=True)
color = plots.get_colors(df_normal, grey=False)

index = list(df_normal.index)

## Display Ranked Strategies vs Normalized Data

In [ ]:
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 12, color = color[col]),
#         marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(
    title={
            'text': "<b>UPSGT Premia Framework</b>",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
            },
    title_font_family="Calibri",
    xaxis = dict(tickfont = dict(size=12)),
#     legend_title="Strategies",
    showlegend=True,
    plot_bgcolor='White'
                 )

fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Premia Metrics</b>",title_font = {"size": 20},
                    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Higher the Better</b>",title_font = {"size": 20},
                 showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

# Ranked Stratagies with Premia Universe

## Import Premia Universe Weekly Data

In [ ]:
premia_uni= pd.read_excel(dm.NEW_DATA+'liq_alts.xlsx', sheet_name='premia_uni',index_col=0)
premia_uni = dm.format_data(premia_uni,freq)
premia_uni.drop(['EQ Size'], axis=1, inplace=True)
df_ret_premia = dm.merge_data_frames(df_returns, premia_uni)

## Display Returns

In [ ]:
fmt.get_returns_styler(df_ret_premia)

## Compute Return Statistics and Premia Metrics

In [ ]:
analytics_dict_pm = {'return_stats':rs.get_return_stats(df_ret_premia,freq), 
                  'premia_metrics':pm.get_premia_metrics(df_ret_premia, freq)}

In [ ]:
@interact
def display_analytics(stats=['return_stats', 'premia_metrics']):
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics_dict_pm)
    if stats == 'premia_metrics':
        return fmt.get_analytics_styler(analytics_dict_pm, stats)

## Get Premia Universe Raw Scores and Normalized Premia Metrics 

In [ ]:
premia_uni_dict = summary.get_norm_premia_metrics(df_ret_premia)

## Display Premia Metrics and Normalized Data

In [ ]:
@interact
def display_premia_metrics_or_normal( stats = ['Premia Metrics','Normalized Data']):
    if stats == 'Premia Metrics':
        return fmt.format_pm_to_normalize(premia_uni_dict[stats])    
    else:
        return fmt.format_normalized_data(premia_uni_dict[stats])

## Set up data to display

In [ ]:
#get normalized data
premia_uni_normal = premia_uni_dict['Normalized Data'].copy()
premia_uni_normal = premia_uni_normal.transpose()


symbols = plots.get_symbols(df_normal, unique=True)
color = plots.get_colors(df_normal, grey=False)

#rearrange columns so thatdefensive universe columns are before our strategies
premia_uni_only = premia_uni_normal[premia_uni.columns]
liq_alts_only = premia_uni_normal[liq_alts.columns]

#get symbols for defensive universe
premia_uni_symbols = plots.get_symbols(premia_uni_only, unique=False)

#merge defensive universe symbols with strategy symbols from above
new_symbols= dm.merge_data_frames(premia_uni_symbols, symbols)

#get colors for defensive universe as grey
premia_uni_color = plots.get_colors(premia_uni_only, grey=True)

#merge defensive universe colors with strategy colors
new_color= dm.merge_data_frames(premia_uni_color, color)


index = list(premia_uni_normal.index)

## Display Ranked Strategies vs Normalized Data

In [ ]:
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in premia_uni_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=premia_uni_normal[col],
        marker=dict(size = 12, color = new_color[col]),
#         marker_symbol = new_symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(
    title={
            'text': "<b>Premia Framework</b>",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
            },
    title_font_family="Calibri",
    titlefont = {"size":24},
        xaxis = dict(tickfont = dict(size=14)),
#     legend_title="Strategies",
    showlegend=False,
    plot_bgcolor='White'
                 )

fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Premia Metrics</b>",title_font = {"size": 20},
                    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Higher the Better</b>",title_font = {"size": 20},
                 showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()
